In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import autoregressive

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 17
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)

556949
556932
['\ufeffThe Project Gute', 'The Project Guten', 'he Project Gutenb', 'e Project Gutenbe', ' Project Gutenber']



torch.Size([556932, 17])


# fit

In [8]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(char2i, i2char)
net.to(device)
performance = net.fit(encoded, save_path = "checkpoints/autoregressive_lstm.pt", progress_bar = 0)
net.save_architecture("architectures/autoregressive_lstm.architecture")

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

Epoch | Train                 | Minutes
      | Loss     | Error Rate |
---------------------------------------
    1 |   1.2144 |     34.259 |     0.8
    2 |   0.9702 |     29.068 |     1.5
    3 |   0.9262 |     28.001 |     2.2
    4 |   0.9010 |     27.378 |     3.0
    5 |   0.8833 |     26.933 |     3.7


In [14]:
# includes all the information about the epoch and the model, useful for reproducibility

performance

,epoch,train_loss,train_error_rate,minutes,learning_rate,weight_decay,model,embedding_dimension,hidden_units,layers,dropout,parameters
0,1,1.214361,34.259445,0.800802,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
1,2,0.970247,29.068192,1.486204,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
2,3,0.926212,28.000849,2.167026,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
3,4,0.900984,27.378410,2.986085,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335
4,5,0.883336,26.933337,3.659008,0.001,0,Autoregressive LSTM,32,128,2,0.0,230335


In [9]:
# the input for testing

net.tensor2text(encoded[:5])

['\ufeffThe Project Gute',
 'The Project Guten',
 'he Project Gutenb',
 'e Project Gutenbe',
 ' Project Gutenber']

# predict

In [10]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
idx, log_probabilities = net.predict(encoded[:5], main_progress_bar = False, progress_bar = 0)

net.tensor2text(idx)

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335



['\ufeffThe Project Gutenberg-tm electronic ',
 'The Project Gutenberg-tm electronic w',
 'he Project Gutenberg-tm electronic wo',
 'e Project Gutenberg-tm electronic wor',
 ' Project Gutenberg-tm electronic work']

# greedy_search

In [11]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([-2.4522, -2.5134, -2.5617, -2.5795, -2.5688], device='cuda:0')


['\ufeffThe Project Gutenberg-tm electronic ',
 'The Project Gutenberg-tm electronic w',
 'he Project Gutenberg-tm electronic wo',
 'e Project Gutenberg-tm electronic wor',
 ' Project Gutenberg-tm electronic work']

# sample

In [12]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.sample(encoded[:5], progress_bar = False)

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([-10.0899, -24.9052, -11.4014, -22.5374,  -9.9133], device='cuda:0')


['\ufeffThe Project Gutenberg-tm to exchange',
 'The Project Gutenberg-tm file wit as ',
 'he Project Gutenberg Literary\n[Footno',
 'e Project Gutenberg" as to avoid a gr',
 ' Project Gutenberg-tm License\n\n      ']

# beam_search

In [8]:
importlib.reload(autoregressive)    
net = autoregressive.load_architecture("architectures/autoregressive_lstm.architecture")
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(encoded[:5], progress_bar = False)

print(log_probabilities)
pprint([net.tensor2text(t) for t in indexes])

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 32
Hidden units: 128
Layers: 2
Dropout: 0.0
Trainable parameters: 230,335

tensor([[-2.4522, -3.0231, -4.5768, -4.8534, -5.7860],
        [-2.5134, -3.1163, -4.6800, -5.7090, -5.9480],
        [-2.5617, -3.1450, -4.7755, -5.6018, -5.9589],
        [-2.5795, -3.2951, -4.8843, -5.8266, -5.9226],
        [-2.5688, -3.4184, -4.8764, -6.4988, -6.7574]], device='cuda:0')
[['\ufeffThe Project Gutenberg-tm electronic ',
  '\ufeffThe Project Gutenberg Literary Archi',
  '\ufeffThe Project Gutenberg-tm electronic\n',
  '\ufeffThe Project Gutenberg-tm trademark, ',
  '\ufeffThe Project Gutenberg-tm trademark. '],
 ['The Project Gutenberg-tm electronic w',
  'The Project Gutenberg Literary Archiv',
  'The Project Gutenberg-tm electronic\nw',
  'The Project Gutenberg-tm trademark, a',
  'The Project Gutenberg Literary Archil'],
 ['he Project Gutenberg-tm electronic wo',
  'he Project Gut